# Load libraries

In [3]:
import pandas as pd
import zipfile
import sqlite3

# unzip compressed data

In [5]:
# path_to_zip = './data/ml-25m.zip'
# with zipfile.ZipFile(path_to_zip,"r") as zip_ref:
#     zip_ref.extractall("./data")

In [6]:
data_sources = {
    # movielens
    'genome_scores'    : './data/ml-25m/genome-scores.csv',
    'genome_tags'      : './data/ml-25m/genome-tags.csv',
    'links'            : './data/ml-25m/links.csv',
#     'movies'           : './data/ml-25m/movies.csv',
#     'ratings'          : './data/ml-25m/ratings.csv',
    'tags'             : './data/ml-25m/tags.csv',
    # imdb
#     'imdb_name_basics' : './data/name.basics.tsv.gz',
    'imdb_title_basics': './data/title.basics.tsv.gz',
#     'imdb_ratings'     : './data/title.ratings.tsv.gz',
#     'imdb_crew'        : './data/title.crew.tsv.gz',
#     'imdb_principals'  : './data/title.principals.tsv.gz',
}

In [7]:
dfs = {}
for name, path in data_sources.items():
    print(dfs.keys(), end='\r')
    separator = '\t' if 'tsv' in path else ','
    dfs[name] = pd.read_csv(path, sep=separator)

/home/chris/anaconda3/envs/py38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
# EDA 
# combined_df = pd.merge(dfs['genome_scores'], dfs['genome_tags'], on='tagId')
# combined_df.tag.value_counts()

In [10]:
# EDA
# combined_df[combined_df.relevance == 1].tag.unique()
# all_ones = dfs['genome_scores'][dfs['genome_scores'].relevance==1]

In [11]:
# EDA
# for name in data_sources:
#     print(name)
#     print(dfs[name].shape)
#     print(dfs[name].head())
#     print()

In [12]:
# filter the movies we need from imdb based on movies present in tags dataset
merged_df = pd.merge(dfs['genome_scores'],
                     dfs['links'],
                     left_on='movieId',
                     right_on='movieId')
imdb_movies_to_keep = merged_df.imdbId.unique()

# properly format in order to join
imdb_ids_to_keep = 'tt' + pd.Series(imdb_movies_to_keep, dtype='str').str.zfill(7)

# filter imdb dataset
df =  dfs['imdb_title_basics']
dfs['imdb_title_basics_to_keep'] = df[df['tconst'].isin(imdb_ids_to_keep)]
dfs['imdb_title_basics_to_keep'].shape

(13795, 9)

In [13]:
table_cols = {
    'tags'         : ['id', 'name'],
    'movies'       : ['id', 'kind', 'primary_title', 'original_title',
                      'release_year', 'runtime_minutes', 'genres'],
    'tag_relevance': ['movie_id', 'tag_id', 'relevance'],
}

In [14]:
# dfs['imdb_title_basics_to_keep'].apply(lambda x: int(x['tconst'][2:]), axis=1)

In [15]:
# dfs['imdb_title_basics_to_keep']['tconst'].copy().str[2:].copy().astype(int)

In [17]:
# set dfs with proper column names
keep_cols = ['id', 'titleType', 'primaryTitle', 'originalTitle', 'startYear', 'runtimeMinutes', 'genres']
# dfs['imdb_title_basics_to_keep'].loc[:, 'id'] = dfs['imdb_title_basics_to_keep'].apply(lambda x: int(x['tconst'][2:]), axis=1)
dfs['imdb_title_basics_to_keep'].loc[:,'id'] = dfs['imdb_title_basics_to_keep']['tconst'].str[2:].astype(int)
dfs['imdb_title_basics_to_keep'] = dfs['imdb_title_basics_to_keep'][keep_cols]

# specify columns to keep
dfs['imdb_title_basics_to_keep'].columns = ['id', 'kind', 'primary_title', 'original_title',
                      'release_year', 'runtime_minutes', 'genres']
dfs['genome_tags'].columns = ['id', 'name']
dfs['genome_scores'].columns = ['movie_id', 'tag_id', 'relevance']

In [18]:
db_name = './movie_sqlite.db'
conn = sqlite3.connect(db_name)
c = conn.cursor()

In [19]:
tags_table = """
CREATE TABLE IF NOT EXISTS tags (
    id integer PRIMARY KEY,
    name text NOT NULL
);
"""

# data from the title.basics.tsv.gz
# TODO: format genre as json or extract out into own table
movies_table = """
CREATE TABLE IF NOT EXISTS movies (
    id text PRIMARY KEY,
    kind text,
    primary_title text,
    original_title text,
    release_year integer,
    runtime_minutes integer,
    genres text
);
"""

# movie_id is the imdb movie db id (not the movie lens movie id)
# to access this movie in a url add 'tt' to the beginning of it
# https://www.imdb.com/title/tt<movie_id>
relevance_table = """
CREATE TABLE IF NOT EXISTS tag_relevance (
    movie_id integer,
    tag_id name text NOT NULL,
    relevance real NOT NULL
);
"""

# TODO: create table containing columns each representing a tagname
# and rows each representing a movie, values being the relavance value
# this will make querying easier for similarity metric

#TODO check out themovieddb.org and see if there is any dataset that may 
# have some information we would want, if so we can join to this dataset as well.


In [20]:
table_sql = {
    'tags'         : tags_table,
    'movies'       : movies_table,
    'tag_relevance': relevance_table
}

table_data_source = {
    'tags'         : 'genome_tags',
    'movies'       : 'imdb_title_basics_to_keep',
    'tag_relevance': 'genome_scores',
}

for table_name in table_sql:
    # create table
    c.execute(table_sql[table_name])
    # populate table with data
    df = dfs[table_data_source[table_name]]
    df.to_sql(table_name, conn, if_exists='append', index=False)

In [ ]:
conn.close()

In [ ]:
# sample sql to get all tag relevance scores for a movie
"""
    select m.primary_title, t.name, tr.relevance
    from movies m
    join tags t
    join tag_relevance tr
    on m.id = tr.movie_id
    and t.id = tr.tag_id
    where m.id = 12
    order by 3;
"""